In [1]:
from dotenv import load_dotenv
from langchain_anthropic import ChatAnthropic
from sentence_transformers import SentenceTransformer
from utils.matching_functions import find_closest_documents

load_dotenv()
model = SentenceTransformer("nomic-ai/nomic-embed-text-v1", trust_remote_code=True)
chat = ChatAnthropic(temperature=0, model_name="claude-3-haiku-20240307")

<All keys matched successfully>


In [2]:
from utils.wikipedia_functions import get_random_page, get_page_links, get_page_content, check_wikipedia_pages_existence

start_page = get_random_page(1)[0]
end_page = "United States"
end_page_full_content = get_page_content(end_page)
print(start_page)

Kokal


In [3]:
start_links = get_page_links(start_page)
checked_links = check_wikipedia_pages_existence(start_links)
valid_links = [link for link in start_links if checked_links[link]]

In [4]:
from agents.chain_functions import get_crawler_chain, get_summarize_chain, get_explain_links_chain, get_broad_links_chain

crawler_chain = get_crawler_chain(chat)
summarize_chain = get_summarize_chain(chat)
explain_links_chain = get_explain_links_chain(chat)
broad_links_chain = get_broad_links_chain(chat)

In [5]:
from agents.answer_functions import answer_broad_links, answer_crawler, answer_explain_links, answer_summarize

end_page_content = answer_summarize(summarize_chain, end_page_full_content)
print(end_page_content)

The United States is a federal republic with 50 states, a federal district, and several territories. It has a diverse population, with a history of European colonization, Native American displacement, and waves of immigration. The country has a large economy, advanced technology, and a prominent role in global affairs. The government has a presidential system with three branches, and the country has a long history of political and cultural development, including the American Revolution, Civil War, and rise as a superpower. The United States is known for its diversity, individualism, and global influence in areas like literature, arts, media, and sports.


In [9]:
current_page = start_page
current_links = valid_links
trajectory = []
for i in range(10):
    trajectory.append(current_page)
    if end_page in current_links:
        print(f"{end_page} in {current_page}'s links \nPage reached in {i+1} iterations!")
        break
    else:
        closest_links = find_closest_documents(model, current_links, end_page, end_page_content)
        broad_links = answer_broad_links(broad_links_chain, current_links, end_page, end_page_content)
        current_links = list(set([link for link in closest_links + broad_links if link in current_links]))
        reasonings = answer_explain_links(explain_links_chain, current_links, end_page, end_page_content)
        current_page, crawler_output = answer_crawler(crawler_chain, current_page, current_links, reasonings, end_page, end_page_content)
        print(f"Crawler Output: {crawler_output}")
        found_links = get_page_links(current_page)
        checked_links = check_wikipedia_pages_existence(found_links)
        current_links = [link for link in found_links if checked_links[link] and link not in trajectory]

Crawler Output: <reasoning>
Based on the provided information and the goal of reaching the 'United States' page, the most relevant link to choose would be 'Pakistan'. As a neighboring country to the United States, exploring the page on Pakistan can provide valuable context and connections that could help navigate the Wikipedia game more effectively. Understanding the relationship between Pakistan and the United States, as well as the broader geopolitical and cultural context, can assist in identifying relevant links and making informed decisions to reach the desired destination.
</reasoning>

<output>Pakistan</output>
United States in Pakistan's links 
Page reached in 2 iterations!
